In [1]:
from datasets import load_dataset
from tqdm import tqdm
dataset = load_dataset("abokbot/wikipedia-first-paragraph")

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
#show first 5 examples
dataset['train'][:5]

{'id': ['12', '25', '39', '290', '303'],
 'url': ['https://en.wikipedia.org/wiki/Anarchism',
  'https://en.wikipedia.org/wiki/Autism',
  'https://en.wikipedia.org/wiki/Albedo',
  'https://en.wikipedia.org/wiki/A',
  'https://en.wikipedia.org/wiki/Alabama'],
 'title': ['Anarchism', 'Autism', 'Albedo', 'A', 'Alabama'],
 'text': ['Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.',
  "Autism is a neurodevelopmental disorder characterized by difficulties with social interaction and communi

In [3]:
len(dataset['train'])

6458670

In [4]:
def extract_sentences(texts):
    first_sentences = []
    second_sentences = []
    third_sentences = []

    for text in texts:
        # Split the text into sentences
        sentences = text.split('. ')
        
        # Extract the first sentence
        first_sentence = sentences[0] if len(sentences) > 0 else ""
        first_sentences.append(first_sentence + '.' if first_sentence else "")

        # Extract the second sentence, if it exists
        second_sentence = sentences[1] if len(sentences) > 1 else ""
        second_sentences.append(second_sentence + '.' if second_sentence else "")
        
        # Extract the third sentence, if it exists
        third_sentence = sentences[2] if len(sentences) > 2 else ""
        third_sentences.append(third_sentence + '.' if third_sentence else "")


    sentences_dict = {
        'first_sentences': first_sentences,
        'second_sentences': second_sentences,
        'third_sentences': third_sentences,
    }
    return sentences_dict

In [5]:
def splitsentences(datarow):
    return extract_sentences(datarow['text'])

In [6]:
splitsentences(dataset['train'][0:2])

{'first_sentences': ['Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy.',
  'Autism is a neurodevelopmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior.'],
 'second_sentences': ['Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful.',
  "Parents often notice signs during the first three years of their child's life."],
 'third_sentences': ['As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism..',
  'These signs often develop gradually, though some autistic children experience regression in their communication and 

In [7]:
new_dataset = dataset['train'].map(splitsentences, batched=True)

Map:   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [8]:
wikipedia_dataset = new_dataset.remove_columns(['text','url','id'])

In [9]:
wikipedia_dataset[:5]

{'title': ['Anarchism', 'Autism', 'Albedo', 'A', 'Alabama'],
 'first_sentences': ['Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy.',
  'Autism is a neurodevelopmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior.',
  'Albedo (; ) is the measure of the diffuse reflection of solar radiation out of the total solar radiation and measured on a scale from 0, corresponding to a black body that absorbs all incident radiation, to 1, corresponding to a body that reflects all incident radiation..',
  'A, or a, is the first letter and the first vowel of the modern English alphabet and the ISO basic Latin alphabet.',
  'Alabama () is a state in the Southeastern region of the United States, bordered by Tennessee to the north; Georgia to the east; Florida and the Gulf of Mexico to the south; and Mississippi to the west.'],
 'second_se

In [10]:
def is_not_empty(row):
    # Check if either first_sentence or second_sentence is empty
    return row['first_sentences'] != "" and row['second_sentences'] != "" and row['third_sentences'] != ""

# Apply the filter
filtered_dataset = wikipedia_dataset.filter(is_not_empty)

Filter:   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [11]:
len(filtered_dataset)

2454569

In [12]:
filtered_dataset.to_csv('wikipedia_three_sentences.csv')

Creating CSV from Arrow format:   0%|          | 0/2455 [00:00<?, ?ba/s]

857868419